#**Potential Lungs Disease Detection Using Chest X-RAY**

## Importing Required Libraries

In [1]:
import numpy as np # For Data processing
import tensorflow.keras as tk  # For Model processing 
import seaborn as sns  # Python data visualization library based on matplotlib
import matplotlib.pyplot as plt  # For Data Visualization
from tensorflow.keras.preprocessing.image import ImageDataGenerator # For Data Preprocessing
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint, TensorBoard #For Getting The Best Model
from tensorflow.keras.optimizers import Adam # Optimizer 
from sklearn.metrics import classification_report,confusion_matrix # Generating Confusion matrix and classification_report of matrix 

## Data Acquisition
###### This is a face mask detection dataset from kaggle authored by jtiptj (https://www.kaggle.com/datasets/jtiptj/chest-xray-pneumoniacovid19tuberculosis) ,The dataset is organized into 3 folders (train, test, val) and contains subfolders for each image category (Normal/Pneumonia/Covid-19/Tuberculosis). A total of 7135 x-ray images are present.


### Download Dataset From Kaggle

In [ ]:
# Install Kaggle Supporting Python Library
!pip install kaggle

In [ ]:
#Mount Google Drive To Get The Kaggle Credentials
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Create A Temporary Storage To Store The Kaggle Credentials Into Root 
! mkdir ~/.kaggle

In [10]:
#Copy The Credentials From Google Drive To Temporary Storage
! cp '/content/drive/MyDrive/Colab Notebooks/Kaggle cred/kaggle.json' ~/.kaggle/kaggle.json

In [ ]:
# Checking For The Credentials File
! ls -a /root/.kaggle

In [17]:
#Changing The Credentials File To Owner Only
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
#Downloading The Dataset From Kaggle 
! kaggle datasets download jtiptj/chest-xray-pneumoniacovid19tuberculosis

In [ ]:
#Unzip The Dataset 
! unzip /content/chest-xray-pneumoniacovid19tuberculosis.zip

###Data Preprocessing

In [21]:
#Assigning Image Folders To Corresponding Variables 
train_dataset_path = '/content/train'
validation_dataset_path ='/content/val'
test_dataset_path = '/content/test'

In [31]:
#Create Augmented Images With Below Variables
training_datagen = ImageDataGenerator(rescale = 1./255, rotation_range = 40,
                                                                  width_shift_range = 0.2,
                                                                  height_shift_range = 0.2,
                                                                  shear_range = 0.2,
                                                                  zoom_range = 0.2,
                                                                  horizontal_flip = True,
                                                                  fill_mode = 'nearest'
                                                                  )
#This generator reads images from train dataset and 
# create batches of augmented image data 
training_data  = training_datagen.flow_from_directory(train_dataset_path
                                                      ,target_size=(200,200),batch_size = 128,class_mode = 'categorical')   

Found 6326 images belonging to 4 classes.


In [32]:
#Creating validation  and test augmented images
# Generating more images using only rescaling
datagen = ImageDataGenerator(rescale = 1./255)
# This generator reads images from validation ,test dataset and 
# create batches of augmented image data 

validation_data  = datagen.flow_from_directory(validation_dataset_path,target_size=(200,200),
                                                                                              batch_size = 128,class_mode = 'categorical')
test_data  = datagen.flow_from_directory(test_dataset_path,target_size=(200,200),
                                                                                              batch_size = 128,class_mode = 'categorical')
 

Found 38 images belonging to 4 classes.
Found 771 images belonging to 4 classes.


In [26]:
# To Display The Dictionary Of Class
training_data.class_indices 

{'COVID19': 0, 'NORMAL': 1, 'PNEUMONIA': 2, 'TURBERCULOSIS': 3}

In [ ]:
# To Display The Augmented Images
def plotImages(image_arr):
    fig, axes  = plt.subplots(1,5,figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip(image_arr, axes):
          ax.imshow(img)
    plt.tight_layout()
    plt.show()
images = [training_data[0][0][0] for i in range(5)]
plotImages(images)